In [86]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pprint
%matplotlib inline

In [2]:
Orders = pd.read_csv('data/Orders.csv') 
Returns = pd.read_csv('data/Returns.csv')

In [3]:
# Merge orders and returns
Returns=Returns.rename(columns = {'Order ID':'Order.ID'})
Orders = pd.merge(Orders, Returns,how='outer',on='Order.ID')
Orders.Returned = Orders.Returned.replace(np.nan,'No')
Orders.Returned = Orders.Returned.map(lambda x:1 if x=='Yes' else 0)
# Convert to date type
Orders['Order.Date'] = pd.to_datetime(Orders['Order.Date'])
Orders['Ship.Date'] = pd.to_datetime(Orders['Ship.Date'])
Orders['Order_Month'] = Orders['Order.Date'].dt.month
Orders['Order_Year'] = Orders['Order.Date'].dt.year
# Convert to float type
Orders.Profit = Orders.Profit.str.replace('$','').str.replace(',','').astype(float)
Orders.Sales = Orders.Sales.str.replace('$','').str.replace(',','').astype(float)
Orders = Orders.rename(columns = {'Region_x':'Order.Region','Region_y':'Return.Region'})

In [4]:
pd.set_option('display.max_columns', 100)
Orders.head(10)

,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Order.Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Returned,Return.Region,Order_Month,Order_Year
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.0,62.15,40.77,High,0,NaN,11,2014
1,40099,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,FUR-BO-5957,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",341.96,2,0.0,54.71,25.27,High,0,NaN,11,2014
2,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.1,-288.77,923.63,Critical,0,NaN,2,2014
3,26339,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,TEC-CO-3594,Technology,Copiers,"Brother Fax and Copier, Laser",344.68,2,0.1,34.42,65.35,Critical,0,NaN,2,2014
4,26340,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,OFF-ST-5697,Office Supplies,Storage,"Rogers Folders, Wire Frame",133.92,5,0.1,-6.03,41.64,Critical,0,NaN,2,2014
5,26342,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,TEC-AC-4152,Technology,Accessories,"Enermax Flash Drive, Erganomic",70.79,2,0.1,25.13,10.48,Critical,0,NaN,2,2014
6,25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.17,9,0.1,919.97,915.49,Medium,0,NaN,10,2014
7,25334,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-3152,Technology,Phones,"Apple Speaker Phone, VoIP",333.15,3,0.1,88.80,71.02,Medium,0,NaN,10,2014
8,25333,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,OFF-FA-5468,Office Supplies,Fasteners,"OIC Paper Clips, Assorted Sizes",64.15,6,0.1,22.03,5.98,Medium,0,NaN,10,2014
9,25331,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,OFF-BI-4828,Office Supplies,Binders,"Ibico Index Tab, Clear",16.04,2,0.1,-1.30,2.25,Medium,0,NaN,10,2014


# Problem 4: Feature Engineering
#### Step 1: Create the dependent variable
- First of all, we need to generate a categorical variable which indicates whether an order has been returned or not.
- ***Hint:*** the returned orders’ IDs are contained in the dataset “returns”

In [5]:
# Finished in the third cell.

#### Step 2:
- Your manager believes that **how long it took the order to ship** would affect whether the customer would return it or not. 
- He wants you to generate a feature which can measure how long it takes the company to process each order.
- ***Hint:*** Process.Time = Ship.Date - Order.Date

In [6]:
Orders['Process_Time'] = Orders['Ship.Date'] - Orders['Order.Date']

#### Step 3:

- If a product has been returned before, it may be returned again. 
- Let us generate a feature indictes how many times the product has been returned before.
- If it never got returned, we just impute using 0.
- ***Hint:*** Group by different Product.ID

In [7]:
temp = Orders.groupby('Product.ID').agg({'Returned':'sum'})
temp=temp.rename(columns = {'Returned':'Return_times'})
temp = temp.reset_index()
Orders = pd.merge(Orders, temp,how='inner',on='Product.ID')

In [8]:
Orders.Process_Time = Orders.Process_Time.dt.days
Orders

,Row.ID,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Order.Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Returned,Return.Region,Order_Month,Order_Year,Process_Time,Return_times
0,40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.00,62.15,40.7700,High,0,NaN,11,2014,2,0
1,36383,CA-2013-KN16390140-41338,2013-03-05,2013-03-09,Standard Class,KN-163901406,Katherine Nockton,Corporate,19120.0,Philadelphia,Pennsylvania,United States,Eastern US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,466.16,7,0.40,-93.23,32.9800,Medium,0,NaN,3,2013,4,0
2,40099,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,FUR-BO-5957,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",341.96,2,0.00,54.71,25.2700,High,0,NaN,11,2014,2,0
3,39335,CA-2015-JC15775140-42186,2015-07-01,2015-07-03,Second Class,JC-157751404,John Castell,Consumer,90032.0,Los Angeles,California,United States,Western US,USCA,FUR-BO-5957,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",436.00,3,0.15,5.13,52.1200,High,0,NaN,7,2015,2,0
4,33020,US-2013-KD16345140-41628,2013-12-20,2013-12-25,Standard Class,KD-163451402,Katherine Ducich,Consumer,60623.0,Chicago,Illinois,United States,Central US,USCA,FUR-BO-5957,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",359.06,3,0.30,-71.81,22.1800,Medium,0,NaN,12,2013,5,0
5,34662,CA-2012-LC17050140-40940,2012-02-01,2012-02-03,First Class,LC-170501404,Liz Carlisle,Consumer,92691.0,Mission Viejo,California,United States,Western US,USCA,FUR-BO-5957,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",290.67,2,0.15,3.42,54.6400,High,0,NaN,2,2012,2,0
6,26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.10,-288.77,923.6300,Critical,0,NaN,2,2014,2,2
7,21264,IN-2015-MB173057-42179,2015-06-24,2015-06-28,Standard Class,MB-173057,Maria Bertelson,Consumer,NaN,Sydney,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",1236.47,3,0.10,-96.26,92.2300,Medium,0,NaN,6,2015,4,2
8,4116,MX-2014-CM1271582-41885,2014-09-03,2014-09-05,First Class,CM-1271582,Craig Molinari,Corporate,NaN,Cancún,Quintana Roo,Mexico,Central America,LATAM,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",976.96,4,0.20,232.00,328.6590,Critical,0,NaN,9,2014,2,2
9,30218,ID-2015-MM1726078-42181,2015-06-26,2015-07-01,Standard Class,MM-1726078,Magdelene Morse,Consumer,NaN,Kuala Lumpur,Kuala Lumpur,Malaysia,Southeastern Asia,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",2747.70,6,0.00,82.26,222.7800,Medium,1,Southeastern Asia,6,2015,5,2


### Problem 5: Fitting Models

- You can use any binary classification method you have learned so far.
- Use 80/20 training and test splits to build your model. 
- Double check the column types before you fit the model.
- Only include useful features. i.e all the `ID`s should be excluded from your training set.
- Note that there are only less than 5% of the orders have been returned, so you should consider using the [createDataPartition](https://www.rdocumentation.org/packages/caret/versions/6.0-80/topics/createDataPartition) function from `caret` package and [StratifiedKfold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn-model-selection-stratifiedkfold) from sklearn when running cross-validation.
- Do forget to `set.seed()` before the spilt to make your result reproducible.
- **Note:** We are not looking for the best tuned model in the lab so don't spend too much time on grid search. Focus on model evaluation and the business use case of each model.

In [203]:
# Find missing data
total = Orders.isnull().sum().sort_values(ascending=False)
percent = (Orders.isnull().sum()/Orders.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(3)

,Total,Percent
Return.Region,49070,0.956717
Postal.Code,41296,0.805147
Return_times,0,0.000000


In [204]:
Orders.columns
# Useful feature
# 'Ship.Mode', 'Segment', 'Order.Region', 'Market', 'Category', 'Sub.Category', 'Quantity', 'Discount', 
# 'Order.Priority', 'Order_Month', 'Order_Year', 'Process_Time', 'Return_times'
# Useless feature
# Row.ID, Order.Date, Ship.Date, Customer.ID, Order.ID, Postal.Code, Product.ID, Return.Region
# Product.Name, 
# Not sure
# 'City', 'State', 'Country', 'Sales', 'Profit', 'Shipping.Cost'

# Target
# 'Returned'

Index(['Row.ID', 'Order.ID', 'Order.Date', 'Ship.Date', 'Ship.Mode',
       'Customer.ID', 'Customer.Name', 'Segment', 'Postal.Code', 'City',
       'State', 'Country', 'Order.Region', 'Market', 'Product.ID', 'Category',
       'Sub.Category', 'Product.Name', 'Sales', 'Quantity', 'Discount',
       'Profit', 'Shipping.Cost', 'Order.Priority', 'Returned',
       'Return.Region', 'Order_Month', 'Order_Year', 'Process_Time',
       'Return_times'],
      dtype='object')

In [194]:
#select_data = Orders[['Ship.Mode','Segment', 'Order.Region', 'Market', 'Category', 'Sub.Category', 'Quantity',\
#                     'Discount','Order.Priority', 'Order_Month', 'Order_Year', 'Process_Time', 'Return_times',\
#                     'City', 'State', 'Country', 'Sales', 'Profit', 'Shipping.Cost']].copy()
select_data = Orders[['Process_Time','Return_times']].copy()
y_return = Orders.Returned.copy()

In [196]:
#select_data.Order_Month = select_data.Order_Month.astype(str)
#select_data.Order_Year = select_data.Order_Year.astype(str)
select_data.Process_Time = select_data.Process_Time.astype(str)

In [197]:
select_data.head(1)

,Process_Time,Return_times
0,2,0


In [198]:
# Removing Features with Low Variance
pprint.pprint(dict(zip(select_data.columns, np.var(select_data, 0))))
# Segment could be remove due to low variance
#select_data.drop('Segment',axis=1,inplace=True)
#import sklearn.feature_selection as fs
#select_data = fs.VarianceThreshold(threshold = 1).fit_transform(select_data)

{'Process_Time': 1.8363692331529518}


In [199]:
cate_col = select_data.dtypes[select_data.dtypes == object].index
num_col = select_data.dtypes[select_data.dtypes != object].index
dummies_drop = [i + '_'+ select_data[i].value_counts().index[0] for i in cate_col]
select_data = pd.get_dummies(select_data).drop(dummies_drop,axis=1)

In [200]:
import sklearn.model_selection as ms
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(C=1e-3, solver='lbfgs',max_iter=100)
np.random.seed(0)
#ms_k3s = ms.StratifiedKFold(n_splits=3)
#for train_idx, val_idx in ms_k3s.split(X=select_data, y=y_return):
#    print('Train:', y_return[train_idx], 'Validation: ', y_return[val_idx])
#X_train, X_test, y_train, y_test = train_test_split(select_data, y_return, test_size=0.2, random_state=42)

stratify_divide = ms.StratifiedKFold(n_splits=5, random_state=0)
scores = ms.cross_val_score(estimator=logistic, X=select_data, y=y_return, cv=stratify_divide)
scores

array([0.93117567, 0.95671671, 0.95671671, 0.95671671, 0.95671671])

In [201]:
from sklearn.linear_model import LogisticRegression


logistic.fit(X_train,y_train)
print(logistic.score(X_train,y_train))
print(logistic.coef_)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning:

lbfgs failed to converge. Increase the number of iterations.



0.9565704815753558
[[ 0.00444411  0.00886257  0.26527088 ... -0.00145927  0.00187797
  -0.0017344 ]]


In [202]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,logistic.predict(X_test))

array([[9820,    0],
       [ 438,    0]])

### Problem 6: Evaluating Models
- What is the best metric to evaluate your model. Is accuracy good for this case?
- Now you have multiple models, which one would you pick? 
- Can you get any clue from the confusion matrix? What is the meaning of precision and recall in this case? Which one do you care the most? How will your model help the manager make decisions?
- **Note:** The last question is open-ended. Your answer could be completely different depending on your understanding of this business problem.

In [ ]:
# Precision. Accuracy is not good for this case. We can get 95% accuracy by guess there is no return
# Precision is zero, and Recall is also 0.
# Precision is the conditional probability that the true returned is returned if the predicted class is returned
# Recall is the fraction observations where the actual class is returned that are begin correctly classified
# Process.Time and Return_times may not influence the if the product be returned or not.